In [1]:
from hed.tools.io_utils import get_file_list, make_file_dict
root_path = 'G:/AttentionShift/AttentionShiftExperiments'
files_bids = get_file_list(root_path, extensions=[".tsv"], name_suffix="_events")
files_eeg = get_file_list(root_path, extensions=[".tsv"], name_suffix="_events_temp")
bids_delete_cols = ['HED', 'response_time', 'stim_file']
dict_bids = make_file_dict(files_bids)
dict_eeg = make_file_dict(files_eeg, indices=(0, -3))

In [2]:
from hed.tools.data_utils import get_new_dataframe
print("Correcting cond_code and trial_type for sub-031_run-01")
correct_file = 'G:/AttentionShift/sub-031_run-01_kay_dung_compare.tsv'
df_correct =  get_new_dataframe(correct_file)

bids_corrected = 'G:/AttentionShift/AttentionShiftExperiments/sub-031/eeg/sub-031_task-AuditoryVisualShift_run-01_events.tsv'
df_bids_corrected = get_new_dataframe(bids_corrected)
df_bids_corrected['trial_type'] = df_correct['trial_type']
df_bids_corrected.to_csv(bids_corrected, sep='\t', index=False)

eeg_corrected = 'G:/AttentionShift/AttentionShiftExperiments/sub-031/eeg/sub-031_task-AuditoryVisualShift_run-01_events_temp.tsv'
df_eeg_corrected = get_new_dataframe(eeg_corrected)
df_eeg_corrected['cond_code'] = df_correct['cond_code']
df_eeg_corrected.to_csv(eeg_corrected, sep='\t', index=False)

Correcting cond_code and trial_type for sub-031_run-01


In [3]:
# from hed.tools import check_match, delete_columns, delete_rows_by_column, get_new_dataframe
#
# print(f"\nBIDS form of the events: {len(files_bids)}")
# for key, file in dict_bids.items():
#     df_bids = get_new_dataframe(file)
#     df_eeg = get_new_dataframe(dict_eeg[key])
#
#     delete_columns(df_bids, bids_delete_cols)
#     df_bids['event_code'] = 'n/a'
#     df_bids['cond_code'] = 'n/a'
#     df_bids['event_code'] = df_eeg['event_code'].astype(str)
#     df_bids['cond_code'] = df_eeg['cond_code'].astype(str)
#     errors = check_match(df_bids['cond_code'], df_bids['trial_type'])
#     if errors:
#         print(f"{key}: cond_code ~= trial_type for {errors}")
#     delete_rows_by_column(df_bids, '199', column_list=['event_code'])
#     filename = file[:-4] + "_temp2.tsv"
#     df_bids.to_csv(filename, sep='\t', index=False)



BIDS form of the events: 54


In [4]:
from hed.tools.io_utils import get_file_list
from hed.tools.key_template import KeyTemplate
print("Outputing the template:")
updated_files = get_file_list(root_path, extensions=[".tsv"], name_suffix="_events_temp2")
key_columns = ['cond_code', 'event_code']
template = KeyTemplate(key_columns)
for file in updated_files:
    template.update(file)
template.resort()
template.print()

Outputing the template:
Counts for key [['cond_code', 'event_code']]:
[1, 1]	2459
[1, 2]	2455
[1, 3]	4620
[1, 4]	4620
[1, 5]	18474
[1, 6]	18475
[1, 7]	192
[1, 8]	192
[1, 9]	96
[1, 10]	96
[1, 11]	766
[1, 12]	766
[1, 13]	384
[1, 14]	382
[1, 201]	5239
[1, 202]	194
[2, 1]	2290
[2, 2]	2293
[2, 3]	4580
[2, 4]	4585
[2, 5]	18314
[2, 6]	18307
[2, 8]	2
[2, 12]	3
[2, 201]	4644
[2, 202]	189
[3, 1]	6954
[3, 2]	6953
[3, 3]	96
[3, 4]	96
[3, 5]	383
[3, 6]	385
[3, 7]	9214
[3, 8]	9214
[3, 9]	4600
[3, 10]	4606
[3, 11]	36782
[3, 12]	36755
[3, 13]	18394
[3, 14]	18397
[3, 201]	19145
[3, 202]	545


In [6]:
import os
from hed.tools.io_utils import get_file_list
from hed.tools.data_utils import make_key

print("Checking the validity of the event codes:")
updated_files = get_file_list(root_path, extensions=[".tsv"], name_suffix="_events_temp2")
for file in updated_files:
    key = make_key(os.path.basename(file), [0, -3])
    df_bids = get_new_dataframe(file)
    focus_mask = df_bids['cond_code'].map(str).isin(['1', '2'])
    shift_mask = df_bids['cond_code'].map(str).isin(['3'])
    focus_event_mask = df_bids['event_code'].map(str).isin(['3', '4', '5', '6'])
    shift_event_mask = df_bids['event_code'].map(str).isin(['7', '8', '9', '10', '11', '12', '13', '14'])
    bad_focus = sum(focus_mask & shift_event_mask)
    if bad_focus:
        print(f"{key} has {bad_focus} shift event codes in a focus condition")
    bad_shift = sum(shift_mask & focus_event_mask)
    if bad_shift:
        print(f"{key} has {bad_shift} focus event codes in a shift condition")

Checking the validity of the event codes:
sub-005_run-01 has 5 shift event codes in a focus condition
sub-008_run-01 has 2874 shift event codes in a focus condition
sub-015_run-01 has 239 focus event codes in a shift condition
sub-036_run-02 has 721 focus event codes in a shift condition
